In [45]:
%load_ext autoreload
%autoreload 2

import cv2 as cv
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from qdanalysis.preprocessing.strokedecomposition import simple_stroke_segment
import pathlib

import tensorflow as tf
import tensorflow.keras as keras

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#to make generated figures easier to view
mpl.rcParams['figure.dpi']=300

In [3]:
#path_name = '../datasets/CERUG'
path_name = './CERUG'
path = pathlib.Path(path_name)
files = [fp for fp in path.iterdir()]
files

[PosixPath('CERUG/Writer0909_03-01.ppm'),
 PosixPath('CERUG/Writer0202_03-02.ppm'),
 PosixPath('CERUG/Writer0202_01.ppm')]

In [4]:
from skimage.morphology import skeletonize
#associative arrays, temp
writers = []
pages = []
page_part = []
strokes = []

for file in path.iterdir():
    filename = file.stem
    print(filename)

    writer_no, page_no, page_part_no = re.search(r'(\d*)_(\d*)-?(\d*)?', filename).groups()
    writers.append(writer_no)
    pages.append(page_no)
    page_part.append(page_part_no)

    #open image and process strokes
    kernel_size = (5, 5)
    image = cv.imread(str(file), cv.IMREAD_GRAYSCALE)
    image = cv.GaussianBlur(image, kernel_size, 1.5)
    outfile = './output/' + filename  + '.png'
    cv.imwrite(outfile, image)

    temp = cv.threshold(image, 0, 1, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    print(temp)
    temp = temp*255
    outfile = './output/' + filename + "_skele_" + '.png'
    
    cv.imwrite(outfile, temp)

    strokes.append(simple_stroke_segment(image))

Writer0909_03-01
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Writer0202_03-02
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Writer0202_01
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [5]:
writer_1_stroke_sz = map(lambda x: x.shape, strokes[0])
np.array(list(writer_1_stroke_sz)).max(axis=0)

array([469,  39])

In [6]:
#https://gist.github.com/IdeaKing/11cf5e146d23c5bb219ba3508cca89ec
from typing import Tuple
def resize_with_pad(image: np.array, 
                    new_shape: Tuple[int, int], 
                    padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
    """Maintains aspect ratio and resizes with padding.
    Params:
        image: Image to be resized.
        new_shape: Expected (width, height) of new image.
        padding_color: Tuple in BGR of padding color
    Returns:
        image: Resized image with padding
    """
    original_shape = (image.shape[1], image.shape[0])
    ratio = float(max(new_shape))/max(original_shape)
    new_size = tuple([int(x*ratio) for x in original_shape])
    image = cv.resize(image, new_size)
    delta_w = new_shape[0] - new_size[0]
    delta_h = new_shape[1] - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    image = cv.copyMakeBorder(image, top, bottom, left, right, cv.BORDER_CONSTANT, value=padding_color)
    return image

In [41]:
size = 64
images_padded = []

image_labels = np.ndarray(shape=(0, 2), dtype=float)
temp_dict = {'0909': [1., 0.], '0202': [0., 1.]}

for idx, stroke_list in enumerate(strokes):
    for stroke in stroke_list:
        if max(stroke.shape) > size:
            continue
        
        image_labels = np.append(image_labels, np.array(temp_dict[writers[idx]]).reshape(1, 2), axis=0)

        stroke_colored = cv.cvtColor(stroke.astype(np.uint8), cv.COLOR_GRAY2RGB)
        stroke_padded = resize_with_pad(stroke_colored, (size, size), padding_color=(0, 0, 0))
        images_padded.append(stroke_padded.astype(float))

In [42]:
images_train = np.array(images_padded)
images_train.shape

(4625, 64, 64, 3)

In [43]:
image_labels.shape

(4625, 2)

In [48]:
from qdanalysis.models import adapt_resnet50

model = adapt_resnet50((size, size, 3), 2)
model.compile(optimizer='nadam', loss=keras.losses.binary_crossentropy, metrics=['accuracy', 'mse'])

In [50]:
epochs = 100
model.fit(images_train, image_labels, epochs=epochs, validation_split=0.2)

Epoch 1/100
116/116 [==============================] - 39s 337ms/step - loss: 0.5424 - accuracy: 0.7449 - mse: 0.1974 - val_loss: 0.1969 - val_accuracy: 1.0000 - val_mse: 0.0064
Epoch 2/100
116/116 [==============================] - 1s 12ms/step - loss: 0.5315 - accuracy: 0.7432 - mse: 0.1924 - val_loss: 0.4538 - val_accuracy: 0.8670 - val_mse: 0.1080
Epoch 3/100
116/116 [==============================] - 1s 12ms/step - loss: 0.5292 - accuracy: 0.7459 - mse: 0.1912 - val_loss: 0.2652 - val_accuracy: 0.9924 - val_mse: 0.0226
Epoch 4/100
116/116 [==============================] - 1s 12ms/step - loss: 0.5166 - accuracy: 0.7516 - mse: 0.1854 - val_loss: 0.1906 - val_accuracy: 0.9924 - val_mse: 0.0119
Epoch 5/100
116/116 [==============================] - 1s 12ms/step - loss: 0.5183 - accuracy: 0.7516 - mse: 0.1870 - val_loss: 0.2875 - val_accuracy: 0.9589 - val_mse: 0.0401
Epoch 6/100
116/116 [==============================] - 1s 12ms/step - loss: 0.5092 - accuracy: 0.7576 - mse: 0.1822 - 